In [22]:
# ---------------------------------------------
# LIMPIEZA DE TODAS LAS COLUMNAS DE UN CSV
# ---------------------------------------------
import pandas as pd
import re

In [23]:
# ---------------------------------------------------
# 2. Cargar CSV de entrada
# ---------------------------------------------------
input_path = "proyecto_brecha.csv"
df = pd.read_csv(input_path, encoding="utf-8", sep=';')

print("Archivo cargado con forma:", df.shape)
print("Columnas detectadas:", df.columns.tolist())

Archivo cargado con forma: (815, 2)
Columnas detectadas: ['NOMBRE DEL PROYECTO', 'BRECHA']


In [24]:
# ---------------------------------------------------
# 1. Función de limpieza básica
# ---------------------------------------------------
def clean_text_basic(text):
    """
    Limpieza básica:
      - Convertir a string
      - Eliminar saltos de línea y tabulaciones
      - Convertir a minúsculas
      - Normalizar acentos
      - Eliminar caracteres no alfanuméricos, dejar puntos por abreviaturas
      - Quitar espacios múltiples
    """
    if pd.isna(text):
        return ""

    text = str(text)

    # Remover saltos de línea y tabulaciones
    text = text.replace("\n", " ").replace("\t", " ").replace("\r", " ")

    # Eliminar caracteres invisibles
    text = re.sub(r"[\x00-\x1F\x7F]", " ", text)

    # Minúsculas
    text = text.lower()

    # Reemplazo manual de acentos
    accents = {
        "á": "a", "é": "e", "í": "i",
        "ó": "o", "ú": "u", "ü": "u", "ñ": "n"
    }
    for a, r in accents.items():
        text = text.replace(a, r)

    # Mantener solo letras, números, espacios y puntos
    text = re.sub(r"[^a-z0-9 .]+", " ", text)

    # Eliminar múltiples espacios
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [25]:
# ---------------------------------------------------
# 3. Aplicar limpieza a TODAS las columnas
# ---------------------------------------------------
for col in df.columns:
    new_col = f"{col}_limpio"
    df[new_col] = df[col].apply(clean_text_basic)

print("\nLimpieza completada. Columnas nuevas generadas:")
print([c for c in df.columns if c.endswith("_limpio")])


Limpieza completada. Columnas nuevas generadas:
['NOMBRE DEL PROYECTO_limpio', 'BRECHA_limpio']


In [26]:
def extract_and_remove_first_word(text):
    if pd.isna(text) or len(str(text).strip()) == 0:
        return "", ""
    parts = str(text).strip().split(' ', 1)
    first_word = parts[0]
    rest_of_text = parts[1] if len(parts) > 1 else ""
    return first_word, rest_of_text

df[['NATURALEZA_PROYECTO', 'NOMBRE DEL PROYECTO_limpio_temp']] = df['NOMBRE DEL PROYECTO_limpio'].apply(lambda x: pd.Series(extract_and_remove_first_word(x)))
df['NOMBRE DEL PROYECTO_limpio'] = df['NOMBRE DEL PROYECTO_limpio_temp']
df = df.drop(columns=['NOMBRE DEL PROYECTO_limpio_temp'])

print("Columna 'NATURALEZA_PROYECTO' creada con éxito y 'NOMBRE DEL PROYECTO_limpio' actualizada.")
print(df[['NOMBRE DEL PROYECTO_limpio', 'NATURALEZA_PROYECTO']].head())

Columna 'NATURALEZA_PROYECTO' creada con éxito y 'NOMBRE DEL PROYECTO_limpio' actualizada.
                          NOMBRE DEL PROYECTO_limpio NATURALEZA_PROYECTO
0  de veredas y reconstruccion de pista en la cal...        construccion
1  de veredas en la avenida puya raymondi distrit...        mejoramiento
2  de la infraestructura vial y peatonal en la av...        mejoramiento
3  de av. jorge basadre grohmann desde parque ind...        mejoramiento
4  del servicio de limpieza publica en el distrit...     fortalecimiento


In [27]:
def extract_and_remove_from_distrito(text):
    if pd.isna(text) or not isinstance(text, str):
        return text, ""

    text_lower = text.lower()
    removed_text = ""
    remaining_text = text.strip()

    keyword_distrito = "distrito"
    keyword_provincia = "provincia"
    # TO IMPROVE: modify to find the word region and the others listed
    # Try to find 'distrito' first
    if keyword_distrito in text_lower:
        index = text_lower.find(keyword_distrito)
        removed_text = text[index:].strip()
        remaining_text = text[:index].strip()
    # If 'distrito' is not found, try to find 'provincia'
    elif keyword_provincia in text_lower:
        index = text_lower.find(keyword_provincia)
        removed_text = text[index:].strip()
        remaining_text = text[:index].strip()

    return remaining_text, removed_text

df[['NOMBRE DEL PROYECTO_limpio', 'UBICACION_GEOGRAFICA']] = df['NOMBRE DEL PROYECTO_limpio'].apply(lambda x: pd.Series(extract_and_remove_from_distrito(x)))

print("Columna 'UBICACION_GEOGRAFICA' creada y 'NOMBRE DEL PROYECTO_limpio' actualizada.")
print(df[['NOMBRE DEL PROYECTO_limpio', 'UBICACION_GEOGRAFICA']].head())

Columna 'UBICACION_GEOGRAFICA' creada y 'NOMBRE DEL PROYECTO_limpio' actualizada.
                          NOMBRE DEL PROYECTO_limpio  \
0  de veredas y reconstruccion de pista en la cal...   
1             de veredas en la avenida puya raymondi   
2  de la infraestructura vial y peatonal en la av...   
3  de av. jorge basadre grohmann desde parque ind...   
4             del servicio de limpieza publica en el   

                                UBICACION_GEOGRAFICA  
0                   distrito de pocollay tacna tacna  
1  distrito de coronel gregorio albarracin lanchi...  
2                   distrito de pocollay tacna tacna  
3                   distrito de pocollay tacna tacna  
4                   distrito de pocollay tacna tacna  


In [28]:
df['BRECHA_limpio'] = df['BRECHA_limpio'].str.replace('porcentaje de ', '', regex=False)
print("Cadena 'porcentaje de ' eliminada de la columna 'BRECHA_limpio'.")
print(df[['BRECHA_limpio']].head())

Cadena 'porcentaje de ' eliminada de la columna 'BRECHA_limpio'.
                                       BRECHA_limpio
0  la poblacion urbana sin acceso a los servicios...
1  la poblacion urbana sin acceso a los servicios...
2  la poblacion urbana sin acceso a los servicios...
3  la poblacion urbana sin acceso a los servicios...
4  poblacion no atendida por un adecuado servicio...


In [29]:

# ---------------------------------------------------
# Exportar CSV final
# ---------------------------------------------------
output_path = "proyecto_limpio.csv"

# Seleccionar solo las columnas limpias y las nuevas
cleaned_columns = [col for col in df.columns if col.endswith('_limpio') or col in ['NATURALEZA_PROYECTO', 'UBICACION_GEOGRAFICA']]
df_cleaned = df[cleaned_columns]

df_cleaned.to_csv(output_path, index=False, encoding="utf-8")

print("\nArchivo limpio guardado en:", output_path)
print("Columnas incluidas en el archivo exportado:", df_cleaned.columns.tolist())


Archivo limpio guardado en: proyecto_limpio.csv
Columnas incluidas en el archivo exportado: ['NOMBRE DEL PROYECTO_limpio', 'BRECHA_limpio', 'NATURALEZA_PROYECTO', 'UBICACION_GEOGRAFICA']
